# Companies in Canada

- 1 million companies lines.

**Must:** 
1. ~~Business Name~~
2. ~~Primary Contact phone~~
3. ~~Mailing address~~
5. ~~Mailing city ~~
6. ~~Website~~
7. ~~Mailing Province~~
8. ~~Business activity~~
9. ~~Mailing Postal~~
8. ~~Industry~~ -- we hard code this.
9. ~~NAICS primary~~ -- we hard code this.
4. Fax
11. Primary Contact name and title
13. Primary Contact email
14. Product /services

**Optional:** \
15. Number of employees \
16. Physical address \
17. Physical city \
18. Year founded \
19. Physical Province \
20. Physical Postal \
21. Sales volume \
22. SIC primary 


**Missing**:
- Primary Contact email
- Primary Contact name
- Fax

- Suggestion: pull email from website.

In [1]:
import scrap as src

from requests import get
from requests.exceptions import RequestException
from contextlib import closing
from bs4 import BeautifulSoup, SoupStrainer
from urllib.parse import urlsplit
import urllib.request

import pandas as pd
import numpy as np

import os, sys, httplib2, json, fire, re, string, requests
from collections import OrderedDict, deque
import re, requests, requests.exceptions

import warnings
warnings.filterwarnings("ignore")


pd.set_option("display.max.columns", None)
pd.set_option("display.max.rows", None)

# Yellow Pages

In [ ]:
class yelllowPages():
    
    def __init__(self, url):
        self.url = url
    
    def fetchAttributes(self):
        
        # title
        titles = src.get_elements(self.url, 
                       search={"class": "listing__name jsMapBubbleName"},
                 tag='h3'
                )
        titles = [i for i in titles if i.isdigit() == False][:-1]
        
        # activity: revisit the length
        activities = src.get_elements(self.url, 
                 tag='article',
                 search = {"class": "listing__details__teaser"}
                )
        activities = [i for i in activities if i !="See more text"]
        activities = [i for i in activities if i !="more..."]
        
        # website: revisit length and alignment
        def get_link(url):
            parser = 'html.parser'  # or 'lxml' (preferred) or 'html5lib', if installed
            resp = urllib.request.urlopen(url)
            soup = BeautifulSoup(resp, parser, from_encoding=resp.info().get_param('charset'))

            links = []
            for link in soup.find_all('a', href=True):
                if "/gourl/" in link['href']:
                    links.append(link['href'])

            return links
        sites = get_link(self.url)
        sites = [i.split("%3A%2F%2F")[-1].split("%")[0] for i in sites]
        
        # physical & mailing addresses
        def address_in4s(url):
    
            parser = 'html.parser'  # or 'lxml' (preferred) or 'html5lib', if installed
            resp = urllib.request.urlopen(url)
            soup = BeautifulSoup(resp, parser, from_encoding=resp.info().get_param('charset'))
            addresses = soup.find_all("span", class_="jsMapBubbleAddress")
            addresses = [addresses[i: i + 4] for i in range(0, len(addresses), 4)]

            return addresses
        
        addresses = address_in4s(self.url)
    
        

In [ ]:
addresses = address_in4s("https://www.yellowpages.ca/search/si/1/Restaurants/Toronto+ON")

In [ ]:
Mailing address (Must) 
Mailing city (Must) 
Mailing Province (Must)
Mailing Postal (Must)
Physical address (Optional)
Physical city (Optional)

In [90]:
for j in range(len(addresses)):
    print([i.getText() for i in addresses[j]])

['56 York St', 'Toronto', 'ON', 'M5J 1S8']
['390 Danforth Ave', 'Toronto', 'ON', 'M4K 1P3']
['420 Danforth Ave', 'Toronto', 'ON', 'M4K 1P3']
['125 Southwood Dr', 'Toronto', 'ON', 'M4E 0B8']
['348 Danforth Ave', 'Toronto', 'ON', 'M4K 1N8']
['421 Dundas St W', 'Toronto', 'ON', 'M5T 2W4']
['2263 Bloor St W', 'Toronto', 'ON', 'M6S 1N8']
['400 Bloor St W', 'Toronto', 'ON', 'M5S 1X5']
['1448 Lawrence Ave E', 'North York', 'ON', 'M4A 2S8']
['602 College St', 'Toronto', 'ON', 'M6G 1B4']
['2473 Dufferin St', 'York', 'ON', 'M6B 3P9']
['232 Wellington St W', 'Toronto', 'ON', 'M5V 3W1']
['2861 Dundas St W', 'Toronto', 'ON', 'M6P 1Y9']
['744 Gerrard St E', 'Toronto', 'ON', 'M4M 1Y3']
['94 Ossington Ave', 'Toronto', 'ON', 'M6J 2Z4']
['2883 Dundas St W', 'Toronto', 'ON', 'M6P 1Y9']
['1522 Bayview Ave', 'Toronto', 'ON', 'M4G 3B4']
['95 Danforth Ave', 'Toronto', 'ON', 'M4K 1N2']
['492 Danforth Ave', 'Toronto', 'ON', 'M4K 1P6']
['193, Danforth Ave', 'Toronto', 'ON', 'M4K 1N2']
['938 College St', 'Toront

In [83]:
help(addresses[0][0])

Help on Tag in module bs4.element object:

class Tag(PageElement)
 |  Tag(parser=None, builder=None, name=None, namespace=None, prefix=None, attrs=None, parent=None, previous=None, is_xml=None, sourceline=None, sourcepos=None, can_be_empty_element=None, cdata_list_attributes=None, preserve_whitespace_tags=None)
 |  
 |  Represents an HTML or XML tag that is part of a parse tree, along
 |  with its attributes and contents.
 |  
 |  When Beautiful Soup parses the markup <b>penguin</b>, it will
 |  create a Tag object representing the <b> tag.
 |  
 |  Method resolution order:
 |      Tag
 |      PageElement
 |      builtins.object
 |  
 |  Methods defined here:
 |  
 |  __bool__(self)
 |      A tag is non-None even if it has no contents.
 |  
 |  __call__(self, *args, **kwargs)
 |      Calling a Tag like a function is the same as calling its
 |      find_all() method. Eg. tag('a') returns a list of all the A tags
 |      found within this tag.
 |  
 |  __contains__(self, x)
 |  
 |  __co

In [6]:
def get_phone(url):
    source_code = requests.get(url)
    plain_text = source_code.content
    soup = BeautifulSoup(plain_text, "lxml")
    links = soup.findAll('a', {'class': 'mlr__item__cta jsMlrMenu'})
    
    phones = []
    for link in links:
        title = link.get('data-phone')
        phones.append(title)
    
    return phones

phones = get_phone("https://www.yellowpages.ca/search/si/1/Restaurants/Toronto+ON")
phones

['647-849-3080',
 '647-846-7210',
 '416-463-3333',
 '416-466-2129',
 '416-466-2129',
 '647-360-4006',
 '416-766-0993',
 '416-967-4295',
 '416-285-8980',
 '647-748-4272',
 '416-787-0491',
 '647-347-2727',
 '647-350-2996',
 '416-462-9965',
 '416-901-7667',
 '647-352-7767',
 '416-489-4899',
 '416-465-0969',
 '416-463-4418',
 '647-352-2799',
 '647-348-1116',
 '647-344-1562',
 '416-531-7818',
 '416-782-3302',
 '416-534-3100',
 '647-340-6142',
 '416-588-0005',
 '647-748-2121',
 '647-351-5128',
 '416-466-2931',
 '416-440-0452',
 '416-597-6912',
 '416-977-2929',
 '416-531-3800',
 '416-368-9000']

In [33]:
def get_email(url):

    unprocessed_urls = deque([url])

    processed_urls = set()

    emails = set()
    url = url

    print("Crawling URL %s" % url)
    response = requests.get(url)

    new_emails = set(re.findall(r"[a-z0-9\.\-+_]+@[a-z0-9\.\-+_]+\.[a-z]+", response.text, re.I))
    new_emails = [mail for mail in new_emails if ".png" not in mail]
    new_emails = [mail for mail in new_emails if ".jpg" not in mail]
    new_emails = [i for i in new_emails if len(i) < 30]
    
    emails.update(new_emails)
    
    return emails

In [34]:
get_email("https://www.bytheway.to/")

Crawling URL https://www.bytheway.to/


{'bythewayto@gmail.com', 'info@bytheway.com'}

### products & services

In [25]:
# parser = 'html.parser'  # or 'lxml' (preferred) or 'html5lib', if installed
# resp = urllib.request.urlopen("https://www.yellowpages.ca/search/si/1/Restaurants/Toronto+ON")
# soup = BeautifulSoup(resp, parser, from_encoding=resp.info().get_param('charset'))

# links = []
# for link in soup.find_all('a', href=True):
#     if "bus/" in link['href']:
#         if "#" not in link["href"]:
#             if "Review" not in link["href"]:
#                 links.append(link['href'])



# links = list(OrderedDict.fromkeys(links))
# links

In [24]:
# src.get_elements("https://www.yellowpages.ca" + 
#                  "/bus/Ontario/Toronto/Bardi-s-Steak-House/156059.html?what=Restaurants&where=Toronto+ON&useContext=true", 
#                  tag='li',
# #                  search = {"class": "listing__name--link listing__link jsListingName"}
#                 )

In [ ]:
parser = 'html.parser'  # or 'lxml' (preferred) or 'html5lib', if installed
resp = urllib.request.urlopen("https://www.yellowpages.ca/search/si/1/Restaurants/Toronto+ON")
soup = BeautifulSoup(resp, parser, from_encoding=resp.info().get_param('charset'))

# soup.find(id="text")
soup.findAll("div", {"class": "#text"})

In [ ]:
src.get_elements("https://www.yellowpages.ca/search/si/1/Restaurants/Toronto+ON", 
                 tag='div',
                 search = {"class": "listing__captext"}
                )

# email, fax, contact person